[![Jupyter Notebook](https://img.shields.io/badge/Source%20on%20GitHub-orange)](https://github.com/laminlabs/lamin-usecases/blob/main/docs/spatial4.ipynb)


# Interactive visualization using Vitessce

SpatialDatasets can be visualized interactively on LaminHub using [Vitessce](https://vitessce.io/).  

Here is [a guide](https://docs.lamin.ai/vitessce) for visualizing spatial data with Vitessce.

This [notebook](https://lamin.ai/laminlabs/lamindata/transform/3uuzDQpqbF8M) adds the Vitessce visualization to [this Xenium dataset](https://lamin.ai/laminlabs/lamindata/artifact/8sPWscz3SICG1D8t) discussed on the previous page.

<img src="https://lamin-site-assets.s3.amazonaws.com/.lamindb/0AMvLfVX9VXVbhUf0000.png" width="900px">